# Challenge Telnet

Ce challenge a pour but d'analysée et comprendre les trames Telnet

Pour commencer, comme pour le challenge précédent on importe bien évidemment la librairie "Scapy" qui serra uttile pour les programmes Python.

In [1]:
from scapy.all import *
telnet = rdpcap("telnet-total.pcapng")

## Introduction au Telnet

Comme pour les trames FTP, Wireshark n'affiche pas toute les trames Telnet avec le nom "Telnet". Certaines sont afficher en TCP comme les SYN, SYN/ACK, ACK et FIN.

On doit donc récupérer toute les trames qui uttilise le port Telnet, soit le 23.

La commande est donc la suivante : `tcp.port == 23 || telnet`

![](https://i.ibb.co/CswFWMy9/Capture-d-cran-2025-06-01-010105.png)

## Observation des trames

Telnet fonctionne différament du FTP, ici on a pas de requêtes applicative qui nous donne a quoi correspond chaque action.

Lorsque l'on regarde la capture, on obtient plein d'information comme notament l'ip du serveur (**10.2.12.16**) ou aussi l'ip du client (**10.2.4.3**).

On peut aussi obtenir des informations sensibles en clair comme le login et mot de passe. Ce ci sont envoyée lettre par lettre car la manière dont fonctionne le telnet est que dès que l'uttilisateur tape un carractère pour le login, celui-ci est directement envoyée au serveur qui lui répond cette lettre comme une manière de lui dire qu'il a bien reçu son information.
Pour le mot-de-passe le serveur lui envoie juste un ACK pour lui prouver qu'il a bien reçu pour pas que le mot-de-passe soit récupérée 

Ainsi on peut obtenir ceci :

`login: marie`

`Password: popins`

Juste après son login l'uttilisateur reçoit certaine données du serveur comme la dernière connexion.

De plus, l'uttilisateur vas réalisée quelque commande après sa connexion. Les voici :

`ls -al`
`cat toto`
`exit`

Pour les deux première commande, celle-ci sont des commande bassique Linux. `ls - all` vas permettre de voir tout les fichier présent dans la racine sélectionnée et `cat toto` permet d'afficher le contenu du fichier toto.
La dernière commande `exit` vas arrêter la connexion telnet.

## Script Python

Après tout ce que l'on a vu, on peut réalisée un script python qui vas récupérée automatiquement dans un fichier les login et mot-de-passe d'une connexion **Telnet** :

In [13]:
def telnet_pcapng(packets):
    state = "waiting_login_prompt"  # états : waiting_login_prompt, reading_login, waiting_password_prompt, reading_password
    login_buffer = ""
    password_buffer = ""

    for pkt in packets:
        if pkt.haslayer(IP) and pkt.haslayer(TCP) and pkt.haslayer(Raw):
            payload = pkt[Raw].load.decode("utf-8", errors="ignore")
            sport = pkt[TCP].sport
            dport = pkt[TCP].dport

            # serveur -> client : on cherche les prompts "login:" ou "password:"
            if sport == 23:  
                if state == "waiting_login_prompt" and "login:" in payload.lower():
                    print("Tentative de connexion Telnet")
                    state = "reading_login"
                    login_buffer = ""

                elif state == "waiting_password_prompt" and "password:" in payload.lower():
                    state = "reading_password"
                    password_buffer = ""

            # client -> serveur : on récupère le login ou password caractère par caractère
            elif dport == 23:
                if state == "reading_login":
                    for c in payload:
                        if c in "\r\n":
                            print(f"Login : {login_buffer}")
                            state = "waiting_password_prompt"
                            break
                        else:
                            login_buffer += c

                elif state == "reading_password":
                    for c in payload:
                        if c in "\r\n":
                            print(f"Mot de passe : {password_buffer}")
                            state = "done"
                            return
                        else:
                            password_buffer += c

telnet_pcapng(telnet)

Tentative de connexion Telnet
Login capturé : marie
Mot de passe capturé : poppins


Le fonctionnement de ce programme est très simple. On regarde toute les trames avec un port source à **23**. Si dans les données envoyée on a un "*login:*" alors on lis tout ce que le client envoie ensuite jusqu'à ce qu'il confirme son login, soit jusqu'a ce qu'il appuie sur la touche entrée, en code ce touche s'appelle "*/r/n*". On fait en suite la même chose pour le mot-de-passe est l'on récupérée donc notre login et mot-de-passe comme convenue.

A partir du code ci-dessus, on peut donc réalisée un programme sniffer qui permet de récupérée login/password d'une connexion **Telnet**.

In [ ]:
def telnet_sniffer(pkt):
    global state, login_buffer, password_buffer

    if pkt.haslayer(IP) and pkt.haslayer(TCP) and pkt.haslayer(Raw):
        payload = pkt[Raw].load.decode("utf-8", errors="ignore")
        src_ip = pkt[IP].src
        dst_ip = pkt[IP].dst
        sport = pkt[TCP].sport
        dport = pkt[TCP].dport

        # serveur -> client : prompts
        if sport == 23:
            if state == "waiting_login_prompt" and "login:" in payload.lower():
                state = "reading_login"
                login_buffer = ""

            elif state == "waiting_password_prompt" and "password:" in payload.lower():
                state = "reading_password"
                password_buffer = ""

        # client -> serveur : saisie login/mdp
        elif dport == 23:
            if state == "reading_login":
                for c in payload:
                    if c in "\r\n":
                        print(f"Login : {login_buffer}")
                        state = "waiting_password_prompt"
                        break
                    else:
                        login_buffer += c

            elif state == "reading_password":
                for c in payload:
                    if c in "\r\n":
                        print(f"Mot de passe : {password_buffer}")
                        state = "done"
                    else:
                        password_buffer += c


if __name__ == "__main__":
    state = "waiting_login_prompt"
    login_buffer = ""
    password_buffer = ""

    print("Démarrage du sniffer Telnet...")
    sniff(filter="tcp port 23", prn=telnet_sniffer, store=False)

Voilà pour la partie **Telnet**

Il nous reste donc un dernier challenge, celui sur **HTTP**